# Подержанные автомобили

Датасет содержит информацию о характеристиках и ценах подержанных автомобилей в некоторой стране

## Импорт библиотек, константы

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
RANDOM_STATE = 42

In [3]:
DATASET_PATH = "https://raw.githubusercontent.com/evgpat/edu_stepik_from_idea_to_mvp/main/datasets/cars.csv"

## Загрузка и обзор данных

### Загрузка

In [4]:
# загрузка данных
df = pd.read_csv(DATASET_PATH)

In [5]:
df.shape

(6999, 13)

### Описание данных

**Целевая переменная**
- `selling_price`: цена продажи, числовая

**Признаки**
- `name` (string): модель автомобиля
- `year` (numeric, int): год выпуска с завода-изготовителя
- `km_driven` (numeric, int): пробег на дату продажи
- `fuel` (categorical: _Diesel_ или _Petrol_, или _CNG_, или _LPG_, или _electric_): тип топлива
- `seller_type` (categorical: _Individual_ или _Dealer_, или _Trustmark Dealer_): продавец
- `transmission` (categorical: _Manual_ или _Automatic_): тип трансмиссии
- `owner` (categorical: _First Owner_ или _Second Owner_, или _Third Owner_, или _Fourth & Above Owner_): какой по счёту хозяин?
- `mileage` (string, по смыслу числовой): пробег, требует предобработки
- `engine` (string, по смыслу числовой): рабочий объем двигателя, требует предобработки
- `max_power` (string, по смыслу числовой): пиковая мощность двигателя, требует предобработки
- `torque` (string, по смыслу числовой, а то и 2): крутящий момент, требует предобработки
- `seats` (numeric, float; по смыслу categorical, int)

In [6]:
# информация от столбцах
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           6999 non-null   object 
 1   year           6999 non-null   int64  
 2   selling_price  6999 non-null   int64  
 3   km_driven      6999 non-null   int64  
 4   fuel           6999 non-null   object 
 5   seller_type    6999 non-null   object 
 6   transmission   6999 non-null   object 
 7   owner          6999 non-null   object 
 8   mileage        6797 non-null   object 
 9   engine         6797 non-null   object 
 10  max_power      6803 non-null   object 
 11  torque         6796 non-null   object 
 12  seats          6797 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 711.0+ KB


In [7]:
# случайные три записи из датасета
df.sample(3)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
3624,Toyota Camry 2.5 Hybrid,2016,2000000,68089,Petrol,Trustmark Dealer,Automatic,First Owner,19.16 kmpl,2494 CC,157.7 bhp,213Nm@ 4500rpm,5.0
1237,Maruti S-Cross 2017-2020 Delta DDiS 200 SH,2016,690000,32000,Diesel,Dealer,Manual,First Owner,25.1 kmpl,1248 CC,88.5 bhp,200Nm@ 1750rpm,5.0
4902,Volkswagen Polo 2015-2019 1.5 TDI Highline Plus,2018,681000,27000,Diesel,Individual,Manual,First Owner,20.14 kmpl,1498 CC,88 bhp,230Nm@ 1500-2500rpm,5.0


In [8]:
df.describe()

,year,selling_price,km_driven,seats
count,6999.000000,6.999000e+03,6.999000e+03,6797.000000
mean,2013.818403,6.395152e+05,6.958462e+04,5.419008
std,4.053095,8.089419e+05,5.772400e+04,0.965767
min,1983.000000,2.999900e+04,1.000000e+00,2.000000
25%,2011.000000,2.549990e+05,3.500000e+04,5.000000
50%,2015.000000,4.500000e+05,6.000000e+04,5.000000
75%,2017.000000,6.750000e+05,9.700000e+04,5.000000
max,2020.000000,1.000000e+07,2.360457e+06,14.000000


Change seats column:

In [9]:
# lost seats are replaced with mean
mean = df['seats'].mean()

df['seats'].fillna(mean, inplace=True)

Change mileage column:

In [10]:

df['mileage'] = df['mileage'].str.replace('km/kg', '')
df['mileage'] = df['mileage'].str.replace('kmpl', '')
df['mileage'] = df['mileage'].str.replace(' ', '')

In [11]:
df['mileage'][2596]

'13.45'

In [12]:
# nan in mileadge are replaced with mean
df = df.astype({"mileage": float})
mean = df['mileage'].mean()

df['mileage'].fillna(mean, inplace=True)

Change max_power:

In [13]:
df['max_power'].value_counts(dropna=False)

74 bhp        330
NaN           196
81.80 bhp     193
88.5 bhp      172
67 bhp        149
             ... 
58.33 bhp       1
170.63 bhp      1
282 bhp         1
149.5 bhp       1
177.5 bhp       1
Name: max_power, Length: 317, dtype: int64

In [14]:
df['max_power'] = df['max_power'].str.replace(' bhp', '')
df['max_power'] = np.where(df['max_power'] == '', np.nan, df['max_power'])

In [15]:
df = df.astype({"max_power": float})
mean = df['max_power'].mean()

df['max_power'].fillna(mean, inplace=True)

Change engine:

In [16]:
df['engine'].value_counts(dropna=False)

1248 CC    885
1197 CC    715
998 CC     393
796 CC     375
2179 CC    330
          ... 
1339 CC      1
1489 CC      1
1422 CC      1
2496 CC      1
1950 CC      1
Name: engine, Length: 121, dtype: int64

In [17]:
df['engine'] = df['engine'].str.replace(' CC', '')
df['engine'] = np.where(df['engine'] == '', np.nan, df['engine'])

In [18]:
df = df.astype({"engine": float})
mean = df['engine'].mean()

df['engine'].fillna(mean, inplace=True)

In [19]:
df = df.drop('torque', axis=1)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           6999 non-null   object 
 1   year           6999 non-null   int64  
 2   selling_price  6999 non-null   int64  
 3   km_driven      6999 non-null   int64  
 4   fuel           6999 non-null   object 
 5   seller_type    6999 non-null   object 
 6   transmission   6999 non-null   object 
 7   owner          6999 non-null   object 
 8   mileage        6999 non-null   float64
 9   engine         6999 non-null   float64
 10  max_power      6999 non-null   float64
 11  seats          6999 non-null   float64
dtypes: float64(4), int64(3), object(5)
memory usage: 656.3+ KB


#Predict the price:

In [21]:
X = df.drop(['selling_price', 'name'], axis=1)

y = df['selling_price'] # target

In [22]:
X.head()

,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats
0,2014,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74.00,5.0
1,2014,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0
2,2010,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90.00,5.0
3,2007,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.20,5.0
4,2017,45000,Petrol,Individual,Manual,First Owner,20.14,1197.0,81.86,5.0


Now we will add four columns according to fuel type:

In [32]:
X['fuel'].value_counts(dropna=False)

KeyError: ignored

In [28]:
X['Diesel'] = X['fuel'].apply(lambda x: 1 if x == 'Diesel' else 0)
X['Petrol'] = X['fuel'].apply(lambda x: 1 if x == 'Petrol' else 0)
X['CNG'] = X['fuel'].apply(lambda x: 1 if x == 'CNG' else 0)
X['LPG']= X['fuel'].apply(lambda x: 1 if x == 'LPG' else 0)

In [30]:
X = X.drop('fuel', axis=1)

In [31]:
X.head()

,year,km_driven,seller_type,transmission,owner,mileage,engine,max_power,seats,Diesel,Petrol,CNG,LPG
0,2014,145500,Individual,Manual,First Owner,23.40,1248.0,74.00,5.0,1,0,0,0
1,2014,120000,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0,1,0,0,0
2,2010,127000,Individual,Manual,First Owner,23.00,1396.0,90.00,5.0,1,0,0,0
3,2007,120000,Individual,Manual,First Owner,16.10,1298.0,88.20,5.0,0,1,0,0
4,2017,45000,Individual,Manual,First Owner,20.14,1197.0,81.86,5.0,0,1,0,0


Format seller type and transmission:

In [33]:
X['seller_type'].value_counts(dropna=False)

Individual          5826
Dealer               967
Trustmark Dealer     206
Name: seller_type, dtype: int64

In [35]:
X['seller_type'] = X['seller_type'].map({'Individual' : 0, 'Dealer' : 1, 'Trustmark Dealer' : 2})

In [38]:
X['transmission'].value_counts(dropna=False)

Manual       6095
Automatic     904
Name: transmission, dtype: int64

In [39]:
X['transmission'] = X['transmission'].map({'Manual' : 0, 'Automatic' : 1})

In [40]:
X.head()

,year,km_driven,seller_type,transmission,owner,mileage,engine,max_power,seats,Diesel,Petrol,CNG,LPG
0,2014,145500,0,0,First Owner,23.40,1248.0,74.00,5.0,1,0,0,0
1,2014,120000,0,0,Second Owner,21.14,1498.0,103.52,5.0,1,0,0,0
2,2010,127000,0,0,First Owner,23.00,1396.0,90.00,5.0,1,0,0,0
3,2007,120000,0,0,First Owner,16.10,1298.0,88.20,5.0,0,1,0,0
4,2017,45000,0,0,First Owner,20.14,1197.0,81.86,5.0,0,1,0,0


Format owner column:

In [41]:
X['owner'].value_counts(dropna=False)

First Owner             4587
Second Owner            1791
Third Owner              473
Fourth & Above Owner     144
Test Drive Car             4
Name: owner, dtype: int64

In [42]:
X['owner'] = X['owner'].map({'Test Drive Car' : 0, 'Fourth & Above Owner' : 1,
                             'Third Owner' : 2, 'Second Owner' : 3, 'First Owner' : 4})

In [43]:
X.head()

,year,km_driven,seller_type,transmission,owner,mileage,engine,max_power,seats,Diesel,Petrol,CNG,LPG
0,2014,145500,0,0,4,23.40,1248.0,74.00,5.0,1,0,0,0
1,2014,120000,0,0,3,21.14,1498.0,103.52,5.0,1,0,0,0
2,2010,127000,0,0,4,23.00,1396.0,90.00,5.0,1,0,0,0
3,2007,120000,0,0,4,16.10,1298.0,88.20,5.0,0,1,0,0
4,2017,45000,0,0,4,20.14,1197.0,81.86,5.0,0,1,0,0


Break into training and test parts:

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train.shape, X_test.shape

((5249, 13), (1750, 13))

In [45]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

In [48]:
reg.fit(X_train, y_train)
reg.coef_, reg.intercept_

(array([ 3.32758851e+04, -1.51546125e+00,  8.21440674e+04,  4.79762972e+05,
         6.28900618e+02,  1.06385136e+04,  7.28056408e+01,  1.31297144e+04,
        -4.08978264e+04, -2.22281161e+04, -1.14233292e+05, -2.03913173e+04,
         1.56852725e+05]),
 -67575453.82596308)

In [49]:
pred = reg.predict(X_test)

In [50]:
pred[:10], y_test[:10]

(array([436526.08095659, 353388.18682277, 418989.87120157, 205166.30928387,
        229566.3749405 , 678829.96123989, 547896.78341252, 792829.28508173,
        747606.33312611, 448488.44944698]),
 6565    300000
 2943    225000
 2024    360000
 263     300000
 4586    250000
 4479    700000
 4881    445000
 3583    515000
 6361    500000
 4108    490000
 Name: selling_price, dtype: int64)

In [47]:
X.head()

,year,km_driven,seller_type,transmission,owner,mileage,engine,max_power,seats,Diesel,Petrol,CNG,LPG
0,2014,145500,0,0,4,23.40,1248.0,74.00,5.0,1,0,0,0
1,2014,120000,0,0,3,21.14,1498.0,103.52,5.0,1,0,0,0
2,2010,127000,0,0,4,23.00,1396.0,90.00,5.0,1,0,0,0
3,2007,120000,0,0,4,16.10,1298.0,88.20,5.0,0,1,0,0
4,2017,45000,0,0,4,20.14,1197.0,81.86,5.0,0,1,0,0
